In [ ]:
import pandas as pd

In [ ]:
registry_path = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_unit_dataset/data/stroke_registry/post_hoc_modified/stroke_registry_post_hoc_modified.xlsx'

In [ ]:
patient_selection_path = '/Users/jk1/temp/opsum_extration_output/high_frequency_data_patient_selection_with_details.csv'

In [ ]:
stroke_df = pd.read_excel(registry_path)

In [ ]:
patient_selection_df = pd.read_csv(patient_selection_path, dtype=str)

In [ ]:
stroke_df.head()

In [ ]:
patient_selection_df.head()

In [ ]:
from preprocessing.utils import create_registry_case_identification_column

patient_selection_df['case_admission_id'] = create_registry_case_identification_column(patient_selection_df)
stroke_df['case_admission_id'] = create_registry_case_identification_column(stroke_df)

In [ ]:
restricted_stroke_df = stroke_df[stroke_df.case_admission_id.isin(patient_selection_df.case_admission_id)]

In [ ]:
# if death in hospital, set mRs to 6
restricted_stroke_df.loc[restricted_stroke_df['Death in hospital'] == 'yes', '3M mRS'] = 6
# if 3M Death and 3M mRS nan, set mrs to 6
restricted_stroke_df.loc[(restricted_stroke_df['3M Death'] == 'yes') &
                                    (restricted_stroke_df['3M mRS'].isna()), '3M mRS'] = 6

In [ ]:
n_total_missing_outcomes =  restricted_stroke_df['3M mRS'].isna().sum()
print(f'Number of total missing outcomes: {n_total_missing_outcomes}, {n_total_missing_outcomes/len(restricted_stroke_df)}; n=  {len(restricted_stroke_df)}')

In [ ]:
# find cids from transfers from France (where Non-Swiss == yes & referral == other hospital)
cids_transfers_from_france = restricted_stroke_df[(restricted_stroke_df['Referral'] == 'Other hospital') & (restricted_stroke_df['Non-Swiss'] == 'yes')]['case_admission_id'].values

In [ ]:
n_missing_outcomes_transfers_from_france = restricted_stroke_df[restricted_stroke_df.case_admission_id.isin(cids_transfers_from_france)]['3M mRS'].isna().sum()
print(f'Number of missing outcomes in transfers from france:', n_missing_outcomes_transfers_from_france, 'for a number of transfers from France:', len(cids_transfers_from_france))

In [ ]:
n_missing_outcomes_for_non_swiss_residents = restricted_stroke_df[restricted_stroke_df['Non-Swiss'] == 'yes']['3M mRS'].isna().sum()
print(f'Number of missing outcomes for non swiss residents:', n_missing_outcomes_for_non_swiss_residents, 'of:', len(restricted_stroke_df[restricted_stroke_df['Non-Swiss'] == 'yes']))

In [ ]:
n_missing_outcomes_all_transfers = restricted_stroke_df[(restricted_stroke_df['Referral'] == 'Other hospital')]['3M mRS'].isna().sum()
print(f'Number of missing outcomes for all transferred patients:', n_missing_outcomes_all_transfers, f'for a total of {len(restricted_stroke_df[restricted_stroke_df["Referral"] == "Other hospital"])} events')

In [ ]:
n_missing_outcomes_in_hospital = restricted_stroke_df[restricted_stroke_df['Referral'] == 'In-hospital event']['3M mRS'].isna().sum()
print(f'Number of missing outcomes for in-hospital events:', n_missing_outcomes_in_hospital, f'for a total of {len(restricted_stroke_df[restricted_stroke_df["Referral"] == "In-hospital event"])} events')

In [ ]:
restricted_stroke_df[(restricted_stroke_df['Non-Swiss'] != 'yes') & (restricted_stroke_df['Referral'] != 'In-hospital event') & (restricted_stroke_df['3M mRS'].isna())]